In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
print("Code Executed")

# **AQUIRE DATA**

In [ ]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
combine = [train_df, test_df]
print("Code Executed")

In [ ]:
print(train_df.columns.values)

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

**Comments**

Numeric : Age (Continous) 

Catogorical : 
               PassengerId' (numeric)
               'HomePlanet' (String)
               'CryoSleep' (bool)
               'Cabin' (char/num/char)
               'Destination' (String)
               'Age' (float)
               'VIP' (Bool)
             'RoomService' (Float)
             'FoodCourt' (Float)
             'ShoppingMall' (Float)
             'Spa' (Float)
             'VRDeck' (Float)
             'Name' (String)
             'Transported' (bool)


In [ ]:
# preview the data
train_df.head()

In [ ]:
# preview the data
train_df.tail()

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
#summarising the columns with missing data
# Shape of training data (num_rows, num_columns)
print(train_df.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])


# observations

* although many have missing data the max is 217 out of 8693, therefore no cols will be dropped for missing data
* 3 unique values in home planet, most come is earth with 54%
* 64% were not in cryo sleep
* 8 people shared one partiicular cabin (G/734/S) why?
* Most people 69% were destined for TRAPPIST-1e. There are 3 values possible for this column
* Only 2% were VIP.
* Some people had the same name. More reasons to drop name columns.
* Haf the people were less tha 27 yo, while 75% < 38yo.
* Many did not pay for room services, spa and all the extra niceties, around 75% paid 0 for Room Service.
* 8693 passengers in total

## Analyze by pivoting features

In [ ]:
train_df[['HomePlanet', 'Transported']].groupby(['HomePlanet'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['CryoSleep', 'Transported']].groupby(['CryoSleep'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['Destination', 'Transported']].groupby(['Destination'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
train_df[['VIP', 'Transported']].groupby(['VIP'], as_index=False).mean().sort_values(by='Transported', ascending=False)

## Analyze by visualizing data

In [ ]:
g = sns.FacetGrid(train_df, col='Transported')
g.map(plt.hist, 'Age', bins=20)

## Decisions

* Most of the data seemsto have some level of correlation to the target except 'RoomService' 'FoodCourt' 'ShoppingMall' 'Spa' 'VRDeck' 'Name'
* Name to be dropped
* Combine the others to one and reevaluate.
* Other data will be split like cabin and passengerID to work individula components of the data while others will be encoded

## Wrangle data

### Correcting by dropping features

Drop name, drop PassengerID later

In [ ]:
#dropping name column
train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

In [ ]:
#creating a group column using gggg of PassengerId
for dataset in combine:
    dataset['Group'] = (np.where(dataset['PassengerId'].str.contains('_'),
                      dataset['PassengerId'].str.split('_').str[0],
                      dataset['PassengerId']))
    
#now drop PassengerID from train data only
train_df = train_df.drop(['PassengerId'], axis=1)    
train_df.head()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
#Splitting Cabin to its 3 components, Deck, Number, Side
train_df[['Deck','Num','Side']] = train_df["Cabin"].str.split("/",expand=True,)
test_df[['Deck','Num','Side']] = test_df["Cabin"].str.split("/",expand=True,)
    
    
train_df.head()

In [ ]:
#dropping column section, as we no longer need it
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1) 
#now drop PassengerID from train data only
train_df.head()

In [ ]:
#combinig the RoomService, FoodCourt... to one column 'Extra'
#train
train_df["Extra"] = train_df["RoomService"] + train_df["FoodCourt"] + train_df["ShoppingMall"] + train_df["Spa"] +train_df["VRDeck"]
train_df = train_df.drop(["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"], axis=1)
#test
test_df["Extra"] = test_df["RoomService"] + test_df["FoodCourt"] + test_df["ShoppingMall"] + test_df["Spa"] +test_df["VRDeck"]
test_df = test_df.drop(["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"], axis=1)
train_df.head()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
train_df.info()

## Filling In Missing Values and Encoding some data

1. Ordinal Encoding
2. Imputation

In [ ]:
#encoding
from sklearn.preprocessing import OrdinalEncoder
#defining cols that will undergo ordinal transformation
good_label_cols = ['HomePlanet', 'CryoSleep','Destination','VIP','Deck','Side']
# Apply ordinal encoder 
ordinal_encoder = OrdinalEncoder()
train_df[good_label_cols] = ordinal_encoder.fit_transform(train_df[good_label_cols])
test_df[good_label_cols] = ordinal_encoder.transform(test_df[good_label_cols])
print("Code Executed")

In [ ]:
train_df.head()

In [ ]:
#now we impute, but first we need to figure out which columns have missing data
#summarising the columns with missing data
# Shape of training data (num_rows, num_columns)
print(train_df.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Next imputation using mode.

1. Imputation

In [ ]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer(strategy='most_frequent') # Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(train_df))
imputed_X_valid = pd.DataFrame(my_imputer.transform(test_df))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = train_df.columns
imputed_X_valid.columns = test_df.columns
train_df = imputed_X_train
test_df = imputed_X_valid

#creating a copy of the train data for correlation analysis
train_copy_corr = train_df.copy() #will be used later
#dropping these columns as they have low corr
test_df = test_df.drop(["Group","VIP"], axis=1)
train_df = train_df.drop(["Group","VIP"], axis=1)

print("Code Executed")

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

Testing to see if there are still any columns with missing data, there shouldnt be.

In [ ]:
#summarising the columns with missing data
# Shape of training data (num_rows, num_columns)
print(train_df.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

In [ ]:
test_df.head()

In [ ]:

train_copy_corr.head()

In [ ]:
train_copy_corr["Transported"][:] = train_copy_corr["Transported"][:].replace({True: 1, False: 0}).astype(object)
train_copy_corr = train_copy_corr.apply(pd.to_numeric) # convert all columns of DataFrame to numerics
train_copy_corr.head()

In [ ]:
train_copy_corr.info()

In [ ]:
train_copy_corr.columns

In [ ]:
#summarising the columns with missing data
# Shape of training data (num_rows, num_columns)
print(train_copy_corr.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_copy_corr.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
#viewing correlation
#viewing correlation between data point
import seaborn as sns
import matplotlib.pyplot as plt    
fig, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(train_copy_corr.corr(), ax=ax, annot=True)

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(train_copy_corr.corr()[['Transported']].sort_values(by='Transported', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Survival Rate', fontdict={'fontsize':18}, pad=16);

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

## Analyze by pivoting features and viz data

Analysis using the new columns
* Deck
* Side
* Group
* Num
* Extra

In [ ]:
#reevaluating correlation between the new data and the target. Pivoting Data first
#Deck
train_df[['Deck', 'Transported']].groupby(['Deck'], as_index=False).mean().sort_values(by='Transported', ascending=False)

In [ ]:
#side
train_df[['Side', 'Transported']].groupby(['Side'], as_index=False).mean().sort_values(by='Transported', ascending=False)

## Model, predict and solve

### Models to evaluate:
- Logistic Regression
- KNN or k-Nearest Neighbors
- Support Vector Machines
- Naive Bayes classifier
- Decision Tree
- Random Forrest
- Perceptron
- Artificial neural network
- RVM or Relevance Vector Machine

In [ ]:
from sklearn.model_selection import train_test_split
Y = train_df["Transported"]
X = train_df.drop("Transported", axis=1)

# Break off validation set from training data
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

In [ ]:
Y_train[:] = Y_train[:].replace({True: 1, False: 0})
Y_valid[:] = Y_valid[:].replace({True: 1, False: 0})
Y_train.head()

In [ ]:
Y_train = Y_train[:].astype(str).astype(int)
Y_valid = Y_valid[:].astype(str).astype(int)
Y_train.head()

In [ ]:

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_valid)
Y_valid = Y_valid.to_numpy() #convert Y_valid to numpy array so you van use to cal score
acc_log = round(logreg.score(X_valid, Y_valid) * 100, 2)
acc_log


In [ ]:
Y_valid

In [ ]:
Y_pred

In [ ]:
coeff_df = pd.DataFrame(X_train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

**Positive coefficients increase the log-odds of the response (and thus increase the probability), and negative coefficients decrease the log-odds of the response (and thus decrease the probability).**

* Create a different version of the notebook where we drop the Extraband as it has very high negative correlation

In [ ]:

# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_valid)
acc_svc = round(svc.score(X_valid, Y_valid) * 100, 2)
acc_svc


In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_valid)
acc_knn = round(knn.score(X_valid, Y_valid) * 100, 2)
acc_knn

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_valid)
acc_gaussian = round(gaussian.score(X_valid, Y_valid) * 100, 2)
acc_gaussian

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_valid)
acc_perceptron = round(perceptron.score(X_valid, Y_valid) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_valid)
acc_linear_svc = round(linear_svc.score(X_valid, Y_valid) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_valid)
acc_sgd = round(sgd.score(X_valid, Y_valid) * 100, 2)
acc_sgd

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_valid)
acc_decision_tree = round(decision_tree.score(X_valid, Y_valid) * 100, 2)
acc_decision_tree

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=250)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_valid)
random_forest.score(X_valid, Y_valid)
acc_random_forest = round(random_forest.score(X_valid, Y_valid) * 100, 2)
acc_random_forest

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

Random Forest will be used

# Prep, Predict, and Submit test data

**Prep test Data**

In [ ]:
#checking to see if test and train data are the same
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.info()
print("_"*40)
test_df.info()

In [ ]:
#create X_test data to be used to predict
X_test = test_df.drop("PassengerId", axis=1)
X_test.head()

**Predict Test Data**

In [ ]:
#using the best performing model

# Random Forest

#random_forest = RandomForestClassifier(n_estimators=250)
#random_forest.fit(X_train, Y_train)
Y_pred_test = random_forest.predict(X_test)
#random_forest.score(X_valid, Y_valid)
#acc_random_forest = round(random_forest.score(X_valid, Y_valid) * 100, 2)
#acc_random_forest
Y_pred_test

**Submit**

In [ ]:
#submit
# Save test predictions to file
output = pd.DataFrame({'PassengerId': test_df["PassengerId"],
                       'Transported': Y_pred_test})
output.head()

In [ ]:
#convert Transported column to bool
output["Transported"] = output["Transported"].replace({1: True, 0: False})
output.head()

In [ ]:
output.to_csv('submission.csv', index=False)
print("Code Executed")